The purpose of this notebook is to showcase some of the most popular profiling tools for Python code and apply it to the lattice point generation in QMCPy.

Author: Sou-Cheng Choi and Joey Coco

Date: Aug 26, 2023

In [1]:
import qmcpy as qp
import numpy as np
import pandas as pd
import time
import matplotlib.pyplot as plt

In [2]:
import os
cwd = os. getcwd()
cwd

'/Users/terrya/Documents/ProgramData/QMCSoftware/demos'

## Define Functions

In [3]:
def gen_points_iid(n, num_ports, seed=None):
    """ generating i.i.d. points """
    np.random.seed(seed)
    # Generate all points randomly from continuous uniform distribution in [0,1)
    points = np.random.random((num_ports, n))
    return points

def gen_points_sobol(n, num_ports, seed=None):
    """ generating randomized Sobol points """
    ld = qp.Sobol(n, seed=seed)  # define the generator
    points = ld.gen_samples(num_ports)  # generate points
    return points
    
def gen_points_lattice(n, num_ports, seed=None):
    """ generating randomized lattice points in serial fashion """
    l = qp.Lattice(dimension=n, seed=seed, is_parallel=False)
    points = l.gen_samples(num_ports) 
    return points
    
def gen_points_lattice_parallel(n, num_ports, seed=None):
    """ generating randomized lattice points in parallel fashion """
    l = qp.Lattice(dimension=n, seed=seed, order='mps', is_parallel=True)
    points = l.gen_samples(num_ports)  
    return points

## Using time or timeit magic

`%timeit`: line-magic 

`%%timeit`: cell-magic 

`%time`: magic function

In [4]:
%timeit _ = gen_points_lattice(50, 2 ** 14) 

38.4 ms ± 578 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [5]:
%time _ = gen_points_lattice_parallel(50, 2 ** 14) 

CPU times: user 70.7 ms, sys: 28.8 ms, total: 99.5 ms
Wall time: 80.4 ms


## Using cprofile

The following prints long outputs

The following -o captures output in .prof file, but does not work with the decorator @profile 

The following is a magic command that gives same outputs in a pop-up window.

In [6]:
%prun gen_points_lattice_parallel(50, 2 ** 14) 

         1285 function calls (1276 primitive calls) in 0.081 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
       64    0.068    0.001    0.068    0.001 {method 'acquire' of '_thread.lock' objects}
        1    0.009    0.009    0.080    0.080 lattice.py:276(gen_samples)
       15    0.002    0.000    0.002    0.000 {built-in method numpy.asanyarray}
      5/3    0.001    0.000    0.003    0.001 {built-in method numpy.core._multiarray_umath.implement_array_function}
        1    0.000    0.000    0.000    0.000 {built-in method io.open}
        1    0.000    0.000    0.000    0.000 {built-in method numpy.fromfile}
        6    0.000    0.000    0.000    0.000 {built-in method _thread.start_new_thread}
        1    0.000    0.000    0.000    0.000 {built-in method posix.stat}
        1    0.000    0.000    0.081    0.081 2452960284.py:20(gen_points_lattice_parallel)
        1    0.000    0.000    0.081    0.081 <string>:1(

## Snakeviz (https://jiffyclub.github.io/snakeviz/)

In [7]:
!pip install snakeviz

The following generates an HTML page in a new browser tab and it is very readable:
http://127.0.0.1:8080/snakeviz/%2FUsers%2Fterrya%2FDocuments%2FProgramData%2FQMCSoftware%2Fworkouts%2Fintegration_examples%2Fout%2Fasian_option_multi_level.prof

To exit the following command, press stop button.

In [8]:
%load_ext snakeviz

In [9]:
%%snakeviz
gen_points_lattice_parallel(50, 2 ** 14) 

 
*** Profile stats marshalled to file '/var/folders/l3/wz4prc5d4kv118mgp30jj3_m0000gn/T/tmparsyczqe'.
Embedding SnakeViz in this document...
<function display at 0x1029b50d0>


## Tuna

Another alternative for visualizing cprofile outputs

In [10]:
!pip install tuna

The following gives a visual report in a separate tab

## Line-by-line profiling of time and memory

First install `line_profiler` 3.4.0 (`https://github.com/rkern/line_profiler`) 

Need to first put `@profile` before each function that we want to do line or memory profiling.

In [11]:
!pip install line_profiler
!pip install -U memory_profiler

The following two commands would need @decorator written to code.

In [12]:
%load_ext line_profiler

In [13]:
%lprun gen_points_lattice_parallel(50, 2 ** 14) 

Timer unit: 1e-09 s

We will see the following pop-up results:

`data.update_data()` takes up 99.8% of the time. So line profile it now:

## Using memory_profiler

Install `memory_profiler` 0.60.0 from `https://pypi.org/project/memory-profiler/`

In [14]:
%load_ext memory_profiler

`%memit`: line magic similar to `%timeit`
`%%memit`: cell magic like `%%timeit`

In [15]:
%memit gen_points_lattice_parallel(50, 2 ** 14) 

peak memory: 360.69 MiB, increment: 2.47 MiB


After running the above memory profiler, we get a pop up again:

The following needs @profile in the function to be profiled

The following tries to save output

In [16]:
from memory_profiler import LogFile
import sys
sys.stdout = LogFile('memory_profile_log', reportIncrementFlag=False)
%mprun gen_points_lattice_parallel(50, 2 ** 14) 

In [17]:
%mprun gen_points_lattice_parallel(50, 2 ** 14) 

To plot memory usage against time, run the following in a command window.